Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Continuous Mountain Car"

In [2]:
from tqdm import tqdm
import numpy as np
import wandb
import gym
from car_model import Car
from mountain_car_agent import MountainCarAgent

In [4]:
from continuous_mountain_car_env_extended import ContinuousMountainCarEnvExtended

# Cambiar render_mode a rgb_array para entrenar/testear
env = ContinuousMountainCarEnvExtended(render_mode='rgb_array')

In [5]:
x_bins = 20  # Number of bins for position
vel_bins = 20  # Number of bins for velocity
action_bins = 5  # Number of discrete actions to sample from
model = Car(env, x_bins, vel_bins, action_bins)

In [6]:
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
agent = MountainCarAgent(model, alpha, gamma)

In [26]:
# Train the agent
num_training_episodes = 1000
epsilon = 0.2
average_training_rewards = agent.train(num_training_episodes, epsilon)
print(f"Average training reward over {num_training_episodes} episodes: {average_training_rewards}")

Training Progress:   0%|          | 0/1000 [00:00<?, ?episode/s]

Training Progress: 100%|██████████| 1000/1000 [00:03<00:00, 250.65episode/s, Episode Reward=-45]

Average training reward over 1000 episodes: -161.604


In [27]:
# Evaluate the agent
num_evaluation_episodes = 100
average_evaluation_rewards = agent.test(num_evaluation_episodes)
print(f"Average evaluation reward over {num_evaluation_episodes} episodes: {average_evaluation_rewards}")

Average evaluation reward over 100 episodes: -33.56


Obtener el estado a partir de la observación

In [28]:
wandb.init(project="mountain_car",
           config={
               'x_bins': x_bins,
               'vel_bins': vel_bins,
               'action_bins': action_bins,
               'alpha': alpha,
               'gamma': gamma,
               'epsilon': epsilon,
           })
    
epsilon_initial = epsilon
for t in range(10):
    train_value = agent.train(100, epsilon_initial)
    eval_value = agent.test(30)
    wandb.log({'train_value': train_value, 'eval_value': eval_value, "t": t})
    epsilon_initial *= 0.9  # Decay epsilon over iterations

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mateogiraz27 (mateogiraz27-ort). Use `wandb login --relogin` to force relogin


Training Progress: 100%|██████████| 100/100 [00:00<00:00, 307.73episode/s, Episode Reward=-40]


In [29]:
import wandb
from car_model import Car
from mountain_car_agent import MountainCarAgent
from continuous_mountain_car_env_extended import ContinuousMountainCarEnvExtended

def sweep():
    wandb.init()
    config = wandb.config

    env = ContinuousMountainCarEnvExtended(render_mode='rgb_array')
    model = Car(env, config.x_bins, config.vel_bins, config.action_bins)
    agent = MountainCarAgent(model, config.alpha, config.gamma)

    base_epsilon = config.epsilon

    for t in range(12):
        train_value = agent.train(500, base_epsilon)
        test_value = agent.test(50)
        wandb.log({"train_value": train_value, "test_value": test_value, "t" : t})
        base_epsilon *= (10-t)/10

sweep_config = {
        'name': 'bayesian-sweep-epsilon-decay',
        'method': 'bayes',
        'metric': {
            'name': 'test_value',
            'goal': 'maximize'
        },
        'parameters': {
            'alpha': {
                'distribution': 'uniform',
                'min': 0.5,
                'max': 0.99
            },
            'epsilon': {
                'distribution': 'uniform',
                'min': 0.5,
                'max': 0.99
            },
            'gamma': {
                'distribution': 'uniform',
                'min': 0.5,
                'max': 0.99
            },
            'action_bins': {
                'distribution': 'int_uniform',
                'max': 100,
                'min': 10
            },
            'vel_bins': {
                'distribution': 'int_uniform',
                'max': 100,
                'min': 10
            },
            'x_bins': {
                'distribution': 'int_uniform',
                'max': 100,
                'min': 10
            }
        }
    }
entity = "mateogiraz27-ort"
project = "mountain_car"
sweep_id = wandb.sweep(sweep_config, entity=entity, project=project)

Create sweep with ID: kvt4c6r7
Sweep URL: https://wandb.ai/mateogiraz27-ort/mountain_car/sweeps/kvt4c6r7


In [30]:
sweep_id = "mgvkiiaf"
wandb.agent(sweep_id, function=sweep, count=100, entity=entity, project=project)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: mtuh3h6m with config:
wandb: 	action_bins: 10
wandb: 	alpha: 0.5349624470038623
wandb: 	epsilon: 0.5111950360798202
wandb: 	gamma: 0.7586095428869182
wandb: 	vel_bins: 47
wandb: 	x_bins: 84


wandb: 🚀 View run glamorous-butterfly-512 at: https://wandb.ai/mateogiraz27-ort/mountain_car/runs/mzk13lz1
wandb: Find logs at: wandb/run-20241213_010839-mzk13lz1/logs


Training Progress: 100%|██████████| 500/500 [00:01<00:00, 324.71episode/s, Episode Reward=-92] 


t,▁▂▂▃▄▄▅▅▆▇▇█
test_value,▄▄▇▄███▁▃███
train_value,▁▅▆▇▇███████
t,11
test_value,-33.06
train_value,-48.896


wandb: Agent Starting Run: fga0p2i5 with config:
wandb: 	action_bins: 17
wandb: 	alpha: 0.6853138323766247
wandb: 	epsilon: 0.5109640314754463
wandb: 	gamma: 0.7399405569303439
wandb: 	vel_bins: 75
wandb: 	x_bins: 87


...

wandb: Agent Starting Run: gtucxebv with config:
wandb: 	action_bins: 13
wandb: 	alpha: 0.5288530799749362
wandb: 	epsilon: 0.9754818270101902
wandb: 	gamma: 0.5567815836172685
wandb: 	vel_bins: 87
wandb: 	x_bins: 26


Training Progress:  73%|███████▎  | 366/500 [00:01<00:00, 203.13episode/s, Episode Reward=-128]

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x1081f8080>> (for post_run_cell), with arguments args (<ExecutionResult object at 10ffd8710, execution_count=30 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 10ffd8e30, raw_cell="sweep_id = "mgvkiiaf"
wandb.agent(sweep_id, functi.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/mateogiraz/Documents/MountainCarContinuous%20%281%29/car-continuous/continuous_mountain_car.ipynb#X13sZmlsZQ%3D%3D> result=None>,),kwargs {}:


Training Progress:   6%|▌         | 31/500 [00:00<00:02, 221.39episode/s, Episode Reward=-80] 

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import wandb
from car_model import Car
from mountain_car_agent import MountainCarAgent
from continuous_mountain_car_env_extended import ContinuousMountainCarEnvExtended

def sweep_v2():
    wandb.init()
    config = wandb.config

    env = ContinuousMountainCarEnvExtended(render_mode='rgb_array')
    model = Car(env, config.x_bins, config.vel_bins, config.action_bins)
    agent = MountainCarAgent(model, config.alpha, config.gamma)

    base_epsilon = config.epsilon

    for t in range(20):
        train_value = agent.train(500, base_epsilon)
        test_value = agent.test(50)
        wandb.log({"train_value": train_value, "test_value": test_value, "t" : t})
        base_epsilon *= (10-t)/10

sweep_config = {
    'name': 'bayesian-sweep-epsilon-decay-v2',
    'method': 'bayes',
    'metric': {
        'name': 'test_value',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'distribution': 'uniform',
            'min': 0.508,
            'max': 0.99
        },
        'epsilon': {
            'distribution': 'uniform',
            'min': 0.503,
            'max': 0.989
        },
        'gamma': {
            'distribution': 'uniform',
            'min': 0.729,
            'max': 0.974
        },
        'action_bins': {
            'distribution': 'int_uniform',
            'min': 10,
            'max': 100
        },
        'vel_bins': {
            'distribution': 'int_uniform',
            'min': 10,
            'max': 25
        },
        'x_bins': {
            'distribution': 'int_uniform',
            'min': 10,
            'max': 24
        }
    }
}
entity = "mateogiraz27-ort"
project = "mountain_car"
sweep_id = wandb.sweep(sweep_config, entity=entity, project=project)

In [6]:
sweep_id = "l6wrf4n4"
wandb.agent(sweep_id, function=sweep_v2, count=100, entity=entity, project=project)

wandb: Agent Starting Run: ohf1cb0f with config:
wandb: 	action_bins: 20
wandb: 	alpha: 0.6718927213010032
wandb: 	epsilon: 0.830565126667738
wandb: 	gamma: 0.8871260539991896
wandb: 	vel_bins: 12
wandb: 	x_bins: 14


Training Progress: 100%|██████████| 500/500 [00:01<00:00, 303.73episode/s, Episode Reward=-37] 


t,▁▂▂▃▄▄▅▅▆▇▇█
test_value,▂▁▁▁▁▆▆▁▁███
train_value,▁▁▁▁▁▂▄▅▆▇██
t,11
test_value,-29.8
train_value,-27.066


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6zd5n5cx with config:
wandb: 	action_bins: 26
wandb: 	alpha: 0.5199549354506932
wandb: 	epsilon: 0.9273838229092624
wandb: 	gamma: 0.8667085989818117
wandb: 	vel_bins: 18
wandb: 	x_bins: 14


Training Progress: 100%|██████████| 500/500 [00:01<00:00, 315.33episode/s, Episode Reward=3]   


...

Training Progress: 100%|██████████| 500/500 [00:01<00:00, 265.28episode/s, Episode Reward=-3]  


t,▁▂▂▃▄▄▅▅▆▇▇█
test_value,▁▁▁▁▁▁▇▇▅▇██
train_value,▁▁▁▁▁▂▅▇▇███
t,11
test_value,-28.08
train_value,-9.772


wandb: Agent Starting Run: 96gj5h68 with config:
wandb: 	action_bins: 27
wandb: 	alpha: 0.709195235908292
wandb: 	epsilon: 0.6145769002543273
wandb: 	gamma: 0.7891815067094485
wandb: 	vel_bins: 12
wandb: 	x_bins: 14


Training Progress:  95%|█████████▍| 473/500 [00:06<00:00, 82.49episode/s, Episode Reward=-1000]

Training Progress:  24%|██▎       | 118/500 [00:02<00:06, 56.59episode/s, Episode Reward=-1000]Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-631 (_run_job), stopped 6272348160)>>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1032, in _bootstrap
    self._bootstrap_inner()
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1077, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1391, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/Users/mateogiraz/Library/Caches/pypoetry/virtualenvs/ai-project-yycSt0xa-py3.12/lib/python3.12/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/Users/mateogiraz/Library/Caches/pypoetry/virtualenvs/ai-project-yycSt0xa-py3.12/lib/python3.12/site-packages/ipy

In [21]:
# ejecutar el agente con los mejores parametros

alpha = 0.96276
epsilon = 0.75616
gamma = 0.88364
action_bins = 83
x_bins = 10
vel_bins = 13

env = ContinuousMountainCarEnvExtended(render_mode='rgb_array')
model = Car(env, x_bins, vel_bins, action_bins)
agent = MountainCarAgent(model, alpha, gamma)

for t in range(20):
    train_value = agent.train(500, epsilon)
    test_value = agent.test(50)
    epsilon *= (10-t)/10
    print(f"Train value: {train_value}, Test value: {test_value}, Epsilon: {epsilon}")

Training Progress: 100%|██████████| 500/500 [00:06<00:00, 76.44episode/s, Episode Reward=-1000]


Train value: -999.748, Test value: -1000.0, Epsilon: 0.75616


Training Progress: 100%|██████████| 500/500 [00:06<00:00, 78.21episode/s, Episode Reward=-1000]


Train value: -1000.0, Test value: -1000.0, Epsilon: 0.680544


Training Progress: 100%|██████████| 500/500 [00:06<00:00, 79.50episode/s, Episode Reward=-1000]


...

Train value: -3.02, Test value: -3.22, Epsilon: -0.0


Training Progress: 100%|██████████| 500/500 [00:01<00:00, 435.90episode/s, Episode Reward=-5] 


Train value: -3.198, Test value: -4.16, Epsilon: 0.0


Training Progress: 100%|██████████| 500/500 [00:01<00:00, 441.37episode/s, Episode Reward=2]  


Train value: -2.868, Test value: -3.24, Epsilon: -0.0


In [20]:
# Guardar el modelo entrenado
import pickle
with open('mountain_car_agent.pkl', 'wb') as f:
    pickle.dump({
        'agent': agent,
        'parameters': {
            'alpha': alpha,
            'epsilon': epsilon,
            'gamma': gamma,
            'action_bins': action_bins,
            'x_bins': x_bins,
            'vel_bins': vel_bins
        }
    }, f)

In [ ]:
with open('mountain_car_agent.pkl', 'rb') as f:
    loaded_agent = pickle.load(f)

    test_value = loaded_agent['agent'].test(500)
    print(f"Test value: {test_value}")


In [7]:
from tqdm import tqd
import wandb
import gym
from car_model import Car
from stochastic_mountain_car_agent import StochasticMountainCarAgent
from continuous_mountain_car_env_extended import ContinuousMountainCarEnvExtended

# ejecutar el agente con los mejores parámetros, ahora usando stochastic q learning
alpha = 0.96276
epsilon = 0.75616
gamma = 0.88364
action_bins = 83
x_bins = 10
vel_bins = 13
log_sample_size = int(np.ceil(np.log(action_bins)))  # tamaño del subconjunto

env = ContinuousMountainCarEnvExtended(render_mode='rgb_array')
model = Car(env, x_bins, vel_bins, action_bins)

# Usar el agente estocástico
agent = StochasticMountainCarAgent(model, alpha, gamma, log_sample_size)

# Entrenar y evaluar el agente
for t in range(20):
    train_value = agent.train(500, epsilon)
    test_value = agent.test(50)
    epsilon *= (10-t)/10
    print(f"Train value: {train_value}, Test value: {test_value}, Epsilon: {epsilon}")


Training Progress: 100%|██████████| 500/500 [05:01<00:00,  1.66episode/s, Episode Reward=-1000]


Train value: -997.298, Test value: -1000.0, Epsilon: 0.75616


Training Progress: 100%|██████████| 500/500 [05:53<00:00,  1.41episode/s, Episode Reward=-1000]


Train value: -1000.0, Test value: -1000.0, Epsilon: 0.680544


Training Progress: 100%|██████████| 500/500 [05:23<00:00,  1.54episode/s, Episode Reward=-1000]


...

Train value: -92.334, Test value: -995.48, Epsilon: -0.0


Training Progress: 100%|██████████| 500/500 [00:15<00:00, 31.59episode/s, Episode Reward=-48]  


Train value: -104.394, Test value: -948.88, Epsilon: 0.0


Training Progress: 100%|██████████| 500/500 [00:08<00:00, 61.46episode/s, Episode Reward=-45] 


Train value: -60.142, Test value: -978.24, Epsilon: -0.0


In [16]:
import wandb
import pickle
from car_model import Car
from stochastic_mountain_car_agent import StochasticMountainCarAgent
from continuous_mountain_car_env_extended import ContinuousMountainCarEnvExtended

def sweep_stochastic():
    wandb.init()
    config = wandb.config

    env = ContinuousMountainCarEnvExtended(render_mode='rgb_array')
    model = Car(env, config.x_bins, config.vel_bins, config.action_bins)

    log_sample_size = int(np.ceil(np.log(config.action_bins)))
    agent = StochasticMountainCarAgent(model, config.alpha, config.gamma, log_sample_size)

    # epsilon base y alpha base para el decay
    base_epsilon = config.epsilon
    base_alpha = config.alpha

    best_test_value = -np.inf
    best_model_path = "mountain_car_stochastic_agent.pkl"


    for t in range(12):
        # alpha decay
        alpha = base_alpha * (1 - t / 12)
        agent.alpha = alpha

        train_value = agent.train(500, base_epsilon)
        test_value = agent.test(50)
        
        wandb.log({
            "train_value": train_value, 
            "test_value": test_value, 
            "epsilon": base_epsilon, 
            "alpha": alpha, 
            "t": t
        })

        base_epsilon *= (10 - t) / 10


        if test_value > best_test_value:
            best_test_value = test_value
            with open(best_model_path, "wb") as f:
                pickle.dump(agent, f)
            print(f"Nuevo mejor modelo guardado con test_value: {best_test_value}")

    artifact = wandb.Artifact("best_model", type="model")
    artifact.add_file(best_model_path)
    wandb.log_artifact(artifact)

sweep_stochastic_confi = {
    'name': 'bayesian-sweep-stochastic-q-learning-alpha-decay',
    'method': 'bayes',
    'metric': {
        'name': 'test_value',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 0.99
        },
        'epsilon': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.9
        },
        'gamma': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 0.99
        },
        'action_bins': {
            'distribution': 'int_uniform',
            'min': 20,
            'max': 100
        },
        'vel_bins': {
            'distribution': 'int_uniform',
            'min': 10,
            'max': 100
        },
        'x_bins': {
            'distribution': 'int_uniform',
            'min': 10,
            'max': 100
        }
    }
}

# Registrar el sweep en WandB
entity = "mateogiraz27-ort"
project = "mountain_car"
sweep_id = wandb.sweep(sweep_stochastic_confi, entity=entity, project=project)


Create sweep with ID: vw754ly1
Sweep URL: https://wandb.ai/mateogiraz27-ort/mountain_car/sweeps/vw754ly1


In [17]:
sweep_id = "5rac7rwz"
wandb.agent(sweep_id, function=sweep_stochastic, count=100, entity=entity, project=project)

wandb: Agent Starting Run: 0eajihq8 with config:
wandb: 	action_bins: 82
wandb: 	alpha: 0.7809047492371026
wandb: 	epsilon: 0.10895606074683065
wandb: 	gamma: 0.5603103932565742
wandb: 	vel_bins: 85
wandb: 	x_bins: 61


Training Progress: 100%|██████████| 500/500 [00:52<00:00,  9.53episode/s, Episode Reward=-1000]


Nuevo mejor modelo guardado con test_value: -991.76


Training Progress: 100%|██████████| 500/500 [00:45<00:00, 10.90episode/s, Episode Reward=-1000]


Nuevo mejor modelo guardado con test_value: -963.54


...

Nuevo mejor modelo guardado con test_value: -298.44


Training Progress: 100%|██████████| 500/500 [00:06<00:00, 73.38episode/s, Episode Reward=-271] 


Nuevo mejor modelo guardado con test_value: -262.22


Training Progress: 100%|██████████| 500/500 [00:02<00:00, 191.97episode/s, Episode Reward=-42] 


alpha,█▇▇▆▅▅▄▄▃▂▂▁
epsilon,██▇▆▅▃▂▁▁▁▁▁
t,▁▂▂▃▄▄▅▅▆▇▇█
test_value,▅▇▇█▆▅▄▁▁▃▂▄
train_value,▁▂▄▆▇▇██████
alpha,0.07826
epsilon,0
t,11
test_value,-429.2
train_value,-46.098


In [16]:
import pickle
from continuous_mountain_car_env_extended import ContinuousMountainCarEnvExtended
from car_model import Car
from mountain_car_agent import MountainCarAgent

# Cargar el agente desde el archivo
with open('mountain_car_stochastic_agent_v2.pkl', 'rb') as f:
    mountain_car_stochastic_agent = pickle.load(f)

with open('mountain_car_agent.pkl', 'rb') as f:
    loaded_agent = pickle.load(f)
    mountain_car_agent = loaded_agent['agent']

# Probamos los agentes
num_test_episodes = 100
average_test_reward = mountain_car_agent.test(num_test_episodes)
average_test_reward_stochastic = mountain_car_stochastic_agent.test(num_test_episodes)
print(f"Average test reward over {num_test_episodes} episodes for stochastic agent: {average_test_reward_stochastic}")
print(f"Average test reward over {num_test_episodes} episodes for agent: {average_test_reward}")

Average test reward over 100 episodes for stochastic agent: -204.39
Average test reward over 100 episodes for agent: -3.17


In [1]:
import wandb
import pickle
from car_model import Car
from stochastic_mountain_car_agent import StochasticMountainCarAgent
from continuous_mountain_car_env_extended import ContinuousMountainCarEnvExtended

def sweep_stochastic_v2():
    wandb.init()
    config = wandb.config

    vel_bins = 58
    x_bins = 58

    env = ContinuousMountainCarEnvExtended(render_mode='rgb_array')
    model = Car(env, x_bins, vel_bins, config.action_bins)

    log_sample_size = int(np.ceil(np.log(config.action_bins)))
    agent = StochasticMountainCarAgent(model, config.alpha, config.gamma, log_sample_size)

    # epsilon base y alpha base para el decay
    base_epsilon = config.epsilon
    base_alpha = config.alpha

    best_test_value = -np.inf
    best_model_path = "mountain_car_stochastic_agent_v2.pkl"


    for t in range(20):
        # alpha decay
        alpha = base_alpha * (1 - t / 20)
        agent.alpha = alpha

        base_epsilon = base_epsilon * (1 - t / 20) 

        train_value = agent.train(500, base_epsilon)
        test_value = agent.test(50)
        
        wandb.log({
            "train_value": train_value, 
            "test_value": test_value, 
            "epsilon": base_epsilon, 
            "alpha": alpha, 
            "t": t
        })

        if test_value > best_test_value:
            best_test_value = test_value
            with open(best_model_path, "wb") as f:
                pickle.dump(agent, f)
            print(f"Nuevo mejor modelo guardado con test_value: {best_test_value}")

    artifact = wandb.Artifact("best_model", type="model")
    artifact.add_file(best_model_path)
    wandb.log_artifact(artifact)

sweep_stochastic_confi = {
    'name': 'bayesian-sweep-stochastic-q-learning-alpha-decay-v2',
    'method': 'bayes',
    'metric': {
        'name': 'test_value',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'distribution': 'uniform',
            'min': 0.83176,
            'max': 0.98606
        },
        'epsilon': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 0.9
        },
        'gamma': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 0.99
        },
        'action_bins': {
            'distribution': 'int_uniform',
            'min': 21,
            'max': 25
        },
    }
}

entity = "mateogiraz27-ort"
project = "mountain_car"
sweep_id = wandb.sweep(sweep_stochastic_confi, entity=entity, project=project)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: xmht62ar
Sweep URL: https://wandb.ai/mateogiraz27-ort/mountain_car/sweeps/xmht62ar


In [3]:
sweep_id = "xmht62ar"
wandb.agent(sweep_id, function=sweep_stochastic_v2, count=50, entity=entity, project=project)

wandb: Agent Starting Run: uiep085m with config:
wandb: 	action_bins: 23
wandb: 	alpha: 0.8615490507982176
wandb: 	epsilon: 0.5230815619024021
wandb: 	gamma: 0.6498593380017677
wandb: Currently logged in as: mateogiraz27 (mateogiraz27-ort). Use `wandb login --relogin` to force relogin


Training Progress: 100%|██████████| 500/500 [00:16<00:00, 30.00episode/s, Episode Reward=-564] 


Nuevo mejor modelo guardado con test_value: -375.58


Training Progress: 100%|██████████| 500/500 [00:10<00:00, 48.47episode/s, Episode Reward=-370] 


Nuevo mejor modelo guardado con test_value: -297.46


...

Nuevo mejor modelo guardado con test_value: -205.3


alpha,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
epsilon,██▇▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁
t,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_value,▂▅▆▆▅▆▃▃▁▁▃▃▄▄▅▆▇▇▇█
train_value,▁▄▆▆▇▇▇▇▇▇██████████
alpha,0.04444
epsilon,0.0
t,19
test_value,-205.3
train_value,-17.758


wandb: Agent Starting Run: 3ajitrn6 with config:
wandb: 	action_bins: 23
wandb: 	alpha: 0.8656082267061916
wandb: 	epsilon: 0.5836129923061331
wandb: 	gamma: 0.7213216151126555


Training Progress:  65%|██████▌   | 326/500 [00:12<00:10, 16.30episode/s, Episode Reward=-769] 

Training Progress:  96%|█████████▌| 479/500 [00:19<00:00, 24.93episode/s, Episode Reward=-847] Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-33 (_run_job), stopped 6264041472)>>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1032, in _bootstrap
    self._bootstrap_inner()
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1077, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1391, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/Users/mateogiraz/Library/Caches/pypoetry/virtualenvs/ai-project-yycSt0xa-py3.12/lib/python3.12/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/Users/mateogiraz/Library/Caches/pypoetry/virtualenvs/ai-project-yycSt0xa-py3.12/lib/python3.12/site-packages/ipyk